In [ ]:
import pandas as pd
import surprise as sp
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import KFold
from surprise import accuracy
#from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
df = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/suprise_df.csv")
df5 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/suprise_5.csv")
dfb = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/suprise_b.csv")
df_weight = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/suprise_weight.csv", index_col=0)

In [ ]:
df_weight['ranking_weight'].describe()


In [ ]:
df_weight.sort_values('ranking_weight', inplace=True)

In [ ]:
a,b,c,d,e,f,g,h = np.array_split(df_weight.ranking_weight, 8)

In [ ]:
h

In [ ]:
df_weight['bins2'] = pd.cut(df_weight.ranking_weight,bins = [0.249561,1.212571,1.318157,1.383652,1.436403,1.483643,1.533510,  1.593169, 2.221476 ],
                            labels=[1,2,3,4,5,6,7,8])



In [ ]:
df_weight['bins'] = pd.cut(df_weight.ranking_weight,bins = [0.249561,1.323002,1.437254,1.537974,2.221475, 1.483643, 1.533510,1.593169,],labels=[1,2,3,4])
df_weight.head()


In [ ]:
eight = df_weight.drop(columns=['ranking_weight', 'bins'])

In [ ]:
new = df_weight.drop(columns='ranking_weight')

In [ ]:
new.isnull().any()

In [ ]:
new['bins'].fillna(1, inplace=True)

In [ ]:
df_weight['bins'].value_counts().sort_index()


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2)
df5_train, df5_test = train_test_split(df5, test_size=0.2)
dfb_train, dfb_test = train_test_split(dfb, test_size=0.2)
weight_train, weight_test = train_test_split(df_weight, test_size=0.2)

e_train, e_test = train_test_split(eight, test_size=0.2)



In [ ]:
df_weight.max()

In [ ]:
# Change it back to countries
#df.iloc[:, 1] = labelencoder.inverse_transform(df.iloc[:, 1])
labelencoder = LabelEncoder()
#df.iloc[:, 1] = labelencoder.fit_transform(df.iloc[:, 1])
df

In [ ]:
df.city.value_counts()

In [ ]:
df_weight

In [ ]:
readere = Reader(rating_scale=(1, 8))
datawe = sp.Dataset.load_from_df(eight[['user', 'city', 'bins2']], readere)

In [ ]:
reader= Reader(rating_scale=(1.0, 3.0))
data = sp.Dataset.load_from_df(df_train[['user_id', 'city', 'rank']], reader)

reader5= Reader(rating_scale=(1.0, 5.0))
data5 = sp.Dataset.load_from_df(df5_train[['user_id', 'city', 'rank']], reader5)

readerb = Reader(rating_scale=(1.0, 5.0))
datab = sp.Dataset.load_from_df(dfb_train[['user_id', 'city', 'rank']], readerb)

In [ ]:
# Using surprise train/test
#trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
benchmark = []
# Iterate over all algorithms
for algorithm in [sp.SVD(), sp.NMF(), sp.NormalPredictor(), sp.KNNBasic(), sp.KNNWithZScore(), sp.CoClustering(),
                 sp.SlopeOne(), sp.KNNBaseline(), sp.KNNWithMeans(), sp.SVDpp(), sp.BaselineOnly()]:
# Perform cross validation
    results = cross_validate(algorithm, datawe, measures=['RMSE'], cv=3, verbose=False)
# Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]],index=['Algorithm']))
    benchmark.append(tmp)

In [ ]:
benchmark = pd.DataFrame(benchmark)
benchmark

### GridSearch CV 

With SVD

In [ ]:
param_grid = {'n_factors': [25, 30, 35, 40, 100], 'n_epochs': [15, 20, 25], 'lr_all': [0.001, 0.003, 0.005, 0.008],               'reg_all': [0.08, 0.1, 0.15, 0.02]}
gs = GridSearchCV(sp.SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(dataw) 
algo = gs.best_estimator['rmse']
print(gs.best_score['rmse']) 
print(gs.best_params['rmse'])

In [ ]:

model = sp.SVD(n_factors=100, n_epochs=25, lr_all=.001, reg_all=.08)
trainset = data.build_full_trainset()
model.fit(trainset)

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = model.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
algo2 = sp.SVD(n_factors= 25, n_epochs = 15, lr_all = .001, reg_all = .08)
kf = KFold(n_splits=3)


for trainset, testset in kf.split(dataw):

    # train and test algorithm.
    algo2.fit(trainset)
    predictions = algo2.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions

In [ ]:
sim = {'name': 'pearson_baseline', 'user_based': True, 'min_support': 5}
algo = sp.KNNBasic(k_min = 2, sim_options = sim)
kf = KFold(n_splits=3)

for trainset, testset in kf.split(dataw):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions

In [ ]:
algo3 = sp.BaselineOnly()
kf = KFold(n_splits=3)


for trainset, testset in kf.split(dataw):

    # train and test algorithm.
    algo3.fit(trainset)
    predictions = algo3.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions